In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import math
import datetime
from tqdm import tqdm, trange 

import warnings
warnings.filterwarnings('ignore')

In [2]:
raw_data_1_df = pd.read_excel('./data/data_1.xlsx')
raw_data_2_df = pd.read_excel('./data/data_2.xlsx')
raw_data_3_df = pd.read_excel('./data/data_3.xlsx')

In [3]:
raw_data_1_df.head()

,时间,Unnamed: 1,Unnamed: 2,时间间隔,GPS车速,加速度,X轴加速度,Y轴加速度,Z轴加速度,经度,纬度,发动机转速,扭矩百分比,瞬时油耗,油门踏板开度,空燃比,发动机负荷百分比,进气流量
0,2017/12/18 13:42:13.000.,2017/12/18 13:42:13,NaN,NaN,0.0,NaN,0.0,-0.396,-0.900,119.367781,25.992420,775,18,58.02,0.0,0.1465,22,2.30
1,2017/12/18 13:42:14.000.,2017/12/18 13:42:14,00:00:01,1.0,0.0,0.0,0.0,-0.378,-0.882,119.367783,25.992418,775,17,60.30,0.0,0.1465,21,2.39
2,2017/12/18 13:42:15.000.,2017/12/18 13:42:15,00:00:01,1.0,0.0,0.0,0.0,-0.396,-0.882,119.367785,25.992416,775,17,55.24,0.0,0.1464,22,2.19
3,2017/12/18 13:42:16.000.,2017/12/18 13:42:16,00:00:01,1.0,0.0,0.0,0.0,-0.378,-0.900,119.367785,25.992416,762,17,55.75,0.0,0.1471,21,2.21
4,2017/12/18 13:42:17.000.,2017/12/18 13:42:17,00:00:01,1.0,0.0,0.0,0.0,-0.396,-0.882,119.367785,25.992416,762,16,56.00,0.0,0.1471,21,2.22


## 数据预处理

### 预处理方案

问题汇总：
    1. 数据是要创建成新的运动学片段数据，那么先前的特征'GPS车速', '加速度', 'X轴加速度',
       'Y轴加速度', 'Z轴加速度', '经度', '纬度', '发动机转速', '扭矩百分比', '瞬时油耗', '油门踏板开度', '空燃比',
       '发动机负荷百分比', '进气流量'都不要了吗
    2. 对于数据填充还是不太理解，需要讲个例子进行理解
    3. 怠速时间的问题，删除超出180秒的怠速记录操作不会对后面的特征（怠速时间）造成影响？
    4. 运动学片段划分原则里，加速度是怎么理解？

数据预处理
    1. 对时间间隔在1分钟以内的数据采用插值法进行填补（只对车速），而对时间间隔超出1分钟的数据不处理
        * 公式讲解的不明确，不知道是用哪个值进行填补
    2. 加减速异常的数据
        * 加速度大于4的数据，用前5秒车速的均值去替换此车速（也可替换为前1秒的车速）  万一时间间隔是5分钟，没有前1秒和前5秒呢
            * 这里有个问题，发现1637的加速度为4.22，但是前5秒车速都为0，后5秒都为加速状态，这样直接赋值前5秒均值会造成数据出问题
        * 加速度小于-8的数据，用前5秒车速的均值去替代
    3. 长时间堵车和断断续续低速行驶情况
        * 选取连续10秒车速大于0小于10的数据，速度替换为0  如果连续15秒都是0~10，那么是15秒速度全替换为0？
    4. 怠速时间
        * 怠速时间超出180秒且车速为0的情况，此时选取连续超出180秒车速为0的数据，保留后180秒数据，剔除前面的，即删减冗余数据


运动学片段划分原则：
    1. 速度从0开始
    2. 包含加速度大于0.1，小于-0.1，大于-0.1小于0.1的情况  （这个加速度到底是分了哪几种情况）
    3. 速度以0结束
    4. 片段时长大于20秒
    5. 划分好片段后，若片段中的时间不连续，则剔除该片段  （数据不是依靠时间流进行的吗？为何会出现时间不连续的情况，因为前面怠速删除？）

特征参数体系构建
    1. 怠速时间   （怠速时间超出180s的都被剔除了，不会影响结果吗？）
    2. 加速时间
    3. 匀速时间
    4. 减速时间
    5. 运行时间    
    6. 运行距离
    7. 最大速度
    
    8. 匀速时间比例
    9. 怠速时间比例
    10. 减速时间比例
    11. 加速时间比例
    12. 速度在0-10的比例
    13. 速度在10-20的比例
    14. 速度在20-30的比例
    15. 速度在30-40的比例
    16. 速度在40-50的比例
    17. 速度在50-60的比例
    18. 速度在60-70的比例
    19. 速度在大于70的比例

### 预处理

#### 为间隔时间在60s以内的数据进行线性填充

In [4]:
def delData(df):
    '''
    @func: 删除不必要数据和更改数据名
    @params: [df] 预训练数据
    @return: [df] 处理后数据
    '''
    df = df.drop(columns=['Unnamed: 1', 'Unnamed: 2','X轴加速度','Y轴加速度', 'Z轴加速度', '经度', 
                            '纬度', '发动机转速', '扭矩百分比', '瞬时油耗', '油门踏板开度', '空燃比',
                                    '发动机负荷百分比', '进气流量'])
    df = df.drop([0])  # 删除第一行
    return df


def fillData(df):
    '''
    @func: 为间隔时间在60s以内的数据进行线性填充
    @params: [df] 预训练数据
    @return: [df] 处理后数据
    '''
    df = delData(df)  # 填充前的数据瘦身
    
    # 利用时间间隔去寻找间隔在1分钟以内的数据
    time_interval = df[df['时间间隔'] > 1.000001]
    time_interval = time_interval[time_interval['时间间隔'] <= 60]
    
    def getAcceVelocity(pre_v, v, time_len):
        '''
        @func: 获取加速度
        @params: [pre_v] 上一个时间点速度
        @params: [v] 当前速度
        @params: [time_len] 时长
        @return: [(v - pre_v) / (time_len * 3.6)] 所得到的当前加速度
        '''
        return (v - pre_v) / (time_len * 3.6)

    def getVelocity(pre_v, a, time_len):
        '''
        @func: 获取当前速度
        @params: [pre_v] 上一个时间点速度
        @params: [a] 加速度
        @params: [time_len] 时长
        @return: [pre_v + a * time_len] 所得到的当前速度
        '''
        return pre_v + a * time_len

    def creatDateframe(last_time, time_len, pre_v, a):
        '''
        @func: 创建索要插入的数据，type为DataFrame
        @params: [last_time] 上一个时间点
        @params: [time_len] 时长
        @params: [pre_v] 上一个时间点速度
        @params: [a] 加速度
        @return: [new_df] 新建数据
        '''
        new_df = pd.DataFrame(columns=['时间', '时间间隔', 'GPS车速', '加速度'])
        for idx in range(time_len):
            v_t = getVelocity(pre_v, a, idx+1)
            time_current = str((last_time + datetime.timedelta(seconds=idx+1)).strftime("%Y/%m/%d %H:%M:%S"))+'.000.'
            new_df = new_df.append({'时间':time_current, '时间间隔':1.0, 'GPS车速':v_t, '加速度':a},ignore_index=True)
        return new_df


    add_time = 0
#     i = 0  # i是轮数限定，用来测试code
    for idx in tqdm(time_interval.index,desc='fillData'):
        idx = idx - 1 + add_time
#         i += 1
        now_time = datetime.datetime.strptime(df.iloc[idx]['时间'][:-5],'%Y/%m/%d %H:%M:%S')
        last_time = datetime.datetime.strptime(df.iloc[idx-1]['时间'][:-5],'%Y/%m/%d %H:%M:%S')
        time_span = now_time - last_time
        time_span = int(''.join(re.findall(r"\d+\.?\d*",str(time_span))))

        pre_v = df.iloc[idx-1]['GPS车速']  # 初始速速
        v = df.iloc[idx]['GPS车速']    # 当前速度
        a = getAcceVelocity(pre_v, v, time_span)   # 加速度

        # 对DataFrame的指定位置插入数据，插入数据会破坏索引，因此需要实时记录插入条数
        new_df = creatDateframe(last_time, time_span-1, pre_v, a)  # a~b秒内只需要添加b-a+1行数据，但是最后还要更新当前行
        add_time += (time_span - 1) 
        df1 = df.iloc[:idx] # 将数据按指定位置插入（好麻烦的插入方法）
        df2 = df.iloc[idx:]
        df2['时间间隔'][df2.index[0]] = 1.0
        df = df1.append(new_df, ignore_index = True).append(df2, ignore_index = True)

#         if i>5:break
    return df

#### 对加速度>5和减速度<-8的数据进行数值修正

In [5]:
def moreAcce_and_lessDec(df):
    '''
    @func: 对加速度>5和减速度<-8的数据进行数值修正
    @params: [df] 预训练数据
    @return: [df] 处理后数据
    '''
    # 查看加速度大于4的数据前面是否存在前5秒的数据
    # i = 0 # examine code
    no_5_seconds = []
    for idx in df[df['加速度']>4].index:
    #     i += 1
        last_5_time = datetime.datetime.strptime(df.iloc[idx-5]['时间'][:-5],'%Y/%m/%d %H:%M:%S')
        now_time = datetime.datetime.strptime(df.iloc[idx]['时间'][:-5],'%Y/%m/%d %H:%M:%S')
        time_span = now_time - last_5_time
        time_span = int(''.join(re.findall(r"\d+\.?\d*",str(time_span))))
        if time_span != 5: no_5_seconds.append(idx)
    #     if i>5 :break


    # 查看加速度小于-8的数据前面是否存在前5秒的数据
    # i = 0 # examine code
    no_8_seconds = []
    for idx in df[df['加速度']<-8].index:
    #    i += 1
        last_5_time = datetime.datetime.strptime(df.iloc[idx-5]['时间'][:-5],'%Y/%m/%d %H:%M:%S')
        now_time = datetime.datetime.strptime(df.iloc[idx]['时间'][:-5],'%Y/%m/%d %H:%M:%S')
        time_span = now_time - last_5_time
        time_span = int(''.join(re.findall(r"\d+\.?\d*",str(time_span))))
        if time_span != 5: no_8_seconds.append(idx)
    #     if i>5 :break

    # 对超加速度数据进行数值修正
    for idx in tqdm(df[df['加速度']>4].index,desc='moreAcce_and_lessDec for maorAcce'):
        if idx in no_5_seconds: 
            df.iloc[idx]['GPS车速'] = df.iloc[idx-1]['GPS车速']
        df['GPS车速'][idx] = np.average(df.iloc[idx-5:idx]['GPS车速'].values)

    # 对超减速度数据进行数值修正
    for idx in tqdm(df[df['加速度']<-8].index,desc='moreAcce_and_lessDec for lessDec'):
        if idx in no_8_seconds: 
            df.iloc[idx]['GPS车速'] = df.iloc[idx-1]['GPS车速']
        df['GPS车速'][idx] = np.average(df.iloc[idx-5:idx]['GPS车速'].values)
        
    return df

#### 堵车情况：连续10秒车速大于0小于10的数据，速度替换为0

In [6]:
def updateTrafficJamDate(df):
    '''
    @func: 对连续10秒车速大于0小于10的数据，速度替换为0
    @params: [df] 预训练数据
    @return: [df] 处理后数据
    '''    
    # 先查出这样的数据
    num = 0
    less_10_stack = []
    for idx in range(len(df)):
        if (df.iloc[idx]['GPS车速'] < 10 and df.iloc[idx]['GPS车速'] > 0):
            less_10_stack.append(idx) # 进栈
            num += 1
        elif num < 10 and num > 0 :  
            del less_10_stack[-num:]  # 对不满足10s的已进栈数据进行退栈操作
            num = 0
        else:
            num = 0

    # 对这些数据速度全部替换为0
    for idx in tqdm(less_10_stack, desc='updateTrafficJamDate'):
        df['GPS车速'][idx] = 0   # 不知道为何，这个速度特别慢，比前面遍历一遍df还慢得多的多

    return df

#### 怠速时间超出180秒且车速为0的情况

In [7]:
def more_180_Seconds_process(df):
    '''
    @func: 选取连续超出180秒车速为0的数据，保留后180秒数据，剔除前面的，即删减冗余数据
    @params: [df] 预训练数据
    @return: [df] 处理后数据
    '''  
    def getIndex(more_180_stack,num):
        '''
        @func: 获取怠速栈中后180s的头索引
        @params: [more_180_stack] 连续怠速180以上的索引栈
        @params: [num] 连
        @return: [df] 处理后数据
        '''
        time_len = 0
        for idx,time_span in more_180_stack:
            time_len += time_span
            if time_len > num-180: return idx

    num = 0 # 记录当前栈内的时长（总的时间间隔）
    more_180_stack = []
    sum_num = 0  # 记录总的删减行数
    num_len = 0  # 记录每次删减的数据行数
    max_epoch = len(df) - 1 # 因为要删减数据，所以设置一个最大轮数
    for idx in tqdm(range(len(df)),desc='more_180_Seconds_process'):
        idx = idx - sum_num
        if idx >= (max_epoch - sum_num): break   # 数组越界处理
#         print('idx:', idx, '   ', 'num:',num,'   ', 'num_len:',num_len,'   ', 'sum_num:',sum_num)
        if df.iloc[idx]['GPS车速'] == 0:  # 进栈
            time_span = round(df.iloc[idx]['时间间隔'],0)
            if time_span > 60: # 如果怠速栈遇到时间间隔>60的，则清空栈，重新寻找（因为最终要生成的是运动学片段）
                more_180_stack = []
                num = 0
                continue
            more_180_stack.append((idx, time_span))
            num += time_span
        elif num > 180: # 大于180，寻找索引，删除数据
            begin_idx = more_180_stack[0][0]
            end_idx = getIndex(more_180_stack, num)
            df = df.drop([i for i in range(begin_idx,end_idx)])
            num_len = end_idx - begin_idx
            sum_num += num_len 
            more_180_stack = []
            num = 0
        else: # 小于180，清空栈
            more_180_stack = []
            num = 0
    return df

## 特征建立

特征参数体系构建
    1. 怠速时间   
    2. 加速时间
    3. 匀速时间
    4. 减速时间
    5. 运行时间   # 一个数据相当于一个时间点数，时间是记录整个运动片段的点数即可
    6. 运行距离
    7. 最大速度
    
    8. 匀速时间比例
    9. 怠速时间比例
    10. 减速时间比例
    11. 加速时间比例
    
    12. 速度在0-10的比例
    13. 速度在10-20的比例
    14. 速度在20-30的比例
    15. 速度在30-40的比例
    16. 速度在40-50的比例
    17. 速度在50-60的比例
    18. 速度在60-70的比例
    19. 速度在大于70的比例
    
    20. 平均速度
    21. 平均加速度
    22. 平均减速度
    23. 速度标准差
    24. 加速度标准差

> 1~7和12~19需要在实时建立，8~11可以后添

In [8]:
def creatFeatureDateframe(df, begin_index, end_index, flag):
    '''
    @func: 根据所获得的运动学片段索引，生成一条运动学记录
    @params: [df] 预训练数据
    @params: [begin_index] 运动学片段的头指针
    @params: [end_index] 运动学片段的尾指针
    @params: [flag] 标志字符，标识是创建表头还是生成数据
    @return: [new_df] 处理后数据
    '''
    new_df = pd.DataFrame(columns=['怠速时间', '加速时间', '匀速时间', '减速时间','运行时间','运行距离',
                                   '最大速度','平均速度','平均加速度','平均减速度','速度标准差','加速度标准差',
                                  '怠速时间比例', '加速时间比例', '匀速时间比例', '减速时间比例',
                                  'v_0_10_ratio','v_10_20_ratio','v_20_30_ratio','v_30_40_ratio',
                                  'v_40_50_ratio','v_50_60_ratio','v_60_70_ratio','more_70_v_ratio'])
    
    if flag: return new_df  # 创建表头数据
    
    time_run = end_index - begin_index + 1  # 总的运行时间
    num_time_idling = 0  # 怠速点数记录
    num_time_acc = 0     # 加速点数记录
    num_time_ave = 0     # 匀速点数记录
    num_time_dec = 0     # 减速点数记录
    max_v = 0   # 最大速度
    sum_v = 0   # 片段中所有速度之和
    sum_a_v = 0  # 所有加速阶段的加速度之和
    sum_d_v = 0  # 所有减速阶段的减速度之和
    sum_std_v = 0 # 速度与均速的之差的平方和
    sum_a_2_v = 0 # 加速度平方和
    v_0_10_num = 0  # 速度数值记录
    v_10_20_num = 0
    v_20_30_num = 0
    v_30_40_num = 0
    v_40_50_num = 0
    v_50_60_num = 0
    v_60_70_num = 0
    more_70_v_num = 0 
    
    for idx in range(begin_index, end_index+1):
        
        if df.iloc[idx]['加速度'] > 0.1:
            num_time_acc += 1
            sum_a_v += df.iloc[idx]['加速度']
            sum_a_2_v += df.iloc[idx]['加速度'] **2
        elif df.iloc[idx]['加速度'] < -0.1:
            num_time_dec += 1
            sum_d_v += df.iloc[idx]['加速度']
        elif df.iloc[idx]['加速度'] == 0:
            num_time_idling += 1
        else:
            num_time_ave += 1
            
        if max_v < df.iloc[idx]['GPS车速']: max_v = df.iloc[idx]['GPS车速']
        sum_v += df.iloc[idx]['GPS车速']
        
        if (df.iloc[idx]['GPS车速'] < 10 and df.iloc[idx]['GPS车速'] > 0):
            v_0_10_num += 1
        elif (df.iloc[idx]['GPS车速'] < 20 and df.iloc[idx]['GPS车速'] > 10):
            v_10_20_num += 1
        elif (df.iloc[idx]['GPS车速'] < 30 and df.iloc[idx]['GPS车速'] > 20):
            v_20_30_num += 1
        elif (df.iloc[idx]['GPS车速'] < 40 and df.iloc[idx]['GPS车速'] > 30):
            v_30_40_num += 1
        elif (df.iloc[idx]['GPS车速'] < 50 and df.iloc[idx]['GPS车速'] > 40):
            v_40_50_num += 1
        elif (df.iloc[idx]['GPS车速'] < 60 and df.iloc[idx]['GPS车速'] > 50):
            v_50_60_num += 1
        elif (df.iloc[idx]['GPS车速'] < 70 and df.iloc[idx]['GPS车速'] > 60):
            v_60_70_num += 1
        else:
            more_70_v_num += 1
            
    ave_v = sum_v / (num_time_acc + num_time_dec + num_time_ave)
    for idx in range(begin_index, end_index+1):
        sum_std_v += (df.iloc[idx]['GPS车速']-ave_v) **2
    
    # 添加数值梳理
    time_idling = num_time_idling
    time_acc = num_time_acc
    time_ave = num_time_ave
    time_dec = num_time_dec
    time_run = time_run
    len_run = sum_v / 3.6
    max_v = max_v
    ave_v = sum_v / (num_time_acc + num_time_dec + num_time_ave)
    if num_time_acc == 0: 
        ave_a_v = sum_a_2_v / (num_time_acc + 1)
    else:
        ave_a_v = sum_a_2_v / num_time_acc
    if num_time_dec == 0: 
        ave_d_v = sum_d_v / (num_time_dec + 1)
    else:
        ave_d_v = sum_d_v / num_time_dec
    std_v = (sum_std_v / (time_run - 1)) **0.5
    std_a_v = (sum_a_2_v / (num_time_acc - 1)) **0.5
    time_idling_rate = time_idling / time_run
    time_acc_rate = time_acc / time_run
    time_ave_rate = time_ave / time_run
    time_dec_rate = time_dec / time_run
    v_0_10_ratio = v_0_10_num / time_run
    v_10_20_ratio = v_10_20_num / time_run
    v_20_30_ratio = v_20_30_num / time_run
    v_30_40_ratio = v_30_40_num / time_run
    v_40_50_ratio = v_40_50_num / time_run
    v_50_60_ratio = v_50_60_num / time_run
    v_60_70_ratio = v_60_70_num / time_run
    more_70_v_ratio = more_70_v_num / time_run
    
    
    # 添加运动学片段数据
    new_df = new_df.append({'怠速时间':time_idling, 
                            '加速时间':time_acc, 
                            '匀速时间':time_ave, 
                            '减速时间':time_dec,
                            '运行时间':time_run,
                            '运行距离':len_run,
                            '最大速度':max_v,
                            '平均速度':ave_v,
                            '平均加速度':ave_a_v,
                            '平均减速度':ave_d_v,
                            '速度标准差':std_v,
                            '加速度标准差':std_a_v,
                            '怠速时间比例':time_idling_rate,
                            '加速时间比例':time_acc_rate, 
                            '匀速时间比例':time_ave_rate,
                            '减速时间比例':time_dec_rate,
                            'v_0_10_ratio':v_0_10_ratio,
                            'v_10_20_ratio':v_10_20_ratio,
                            'v_20_30_ratio':v_20_30_ratio,
                            'v_30_40_ratio':v_30_40_ratio,
                            'v_40_50_ratio':v_40_50_ratio,
                            'v_50_60_ratio':v_50_60_ratio,
                            'v_60_70_ratio':v_60_70_ratio,
                            'more_70_v_ratio':more_70_v_ratio},ignore_index=True)
    return new_df
    
    

def searctKinematicSegment(df):
    '''
    @func: 在训练数据中搜索合适的运动学片段，让后创建新的数据表
    @params: [df] 预训练数据
    @return: [feature_df] 生成的运动学片段新数据表
    '''
    begin_index, end_index = 0, 0
    pre_v = 0  # 上一个运动点的速度
    tag = 0 # 0表示新的运动学片段还没开始，遇0则开始， 1表示运动已开始，遇1也是继续循环
    feature_df = creatFeatureDateframe(df, 0, 0, 1)  # 创建表头
    feature_idx = 0
    for idx in tqdm(range(len(df)),desc='searctKinematicSegment'):
        # 如果时间间隔 > 60秒，则放弃正在进行的运动学片段，开始重新寻找新的运动学片段
        if idx == 0:
            interval = datetime.datetime.strptime(df.iloc[idx]['时间'][:-5],'%Y/%m/%d %H:%M:%S') \
                            - datetime.datetime.strptime(df.iloc[idx]['时间'][:-5],'%Y/%m/%d %H:%M:%S')
        else:
            interval = datetime.datetime.strptime(df.iloc[idx]['时间'][:-5],'%Y/%m/%d %H:%M:%S') \
                            - datetime.datetime.strptime(df.iloc[idx-1]['时间'][:-5],'%Y/%m/%d %H:%M:%S')
        if(int(''.join(re.findall(r"\d+\.?\d*",str(interval)))) > 60):
            pre_v = df.iloc[idx]['GPS车速']
            tag = 0
        
        # 运动学片段的判定部分
        if(df.iloc[idx]['GPS车速'] == 0 and tag == 1):  
            if pre_v!=0:  # 则构成了一个运动学片段
                end_index = idx
                now_time = datetime.datetime.strptime(df.iloc[idx]['时间'][:-5],'%Y/%m/%d %H:%M:%S')
                time_span = now_time - last_time
                time_span = int(''.join(re.findall(r"\d+\.?\d*",str(time_span))))
                if time_span > 20:  # 对运动时间在20秒以上的，创建运动学片段数据，并添加
                    new_df = creatFeatureDateframe(df, begin_index, end_index, 0)
                    feature_df = feature_df.append(new_df, ignore_index = True)
                    feature_idx += 1
                else:
                    begin_index = idx
                    last_time = datetime.datetime.strptime(df.iloc[idx]['时间'][:-5],'%Y/%m/%d %H:%M:%S')
                tag = 0  
                pre_v = df.iloc[idx]['GPS车速']
        elif (df.iloc[idx]['GPS车速'] == 0 and tag == 0):  # 新的运动学片段开始
            begin_index = idx
            last_time = datetime.datetime.strptime(df.iloc[idx]['时间'][:-5],'%Y/%m/%d %H:%M:%S')
            pre_v = df.iloc[idx]['GPS车速']
            tag = 1
        elif (df.iloc[idx]['GPS车速'] != 0 and tag == 0):  # 特殊情况，继续循环
            pre_v = df.iloc[idx]['GPS车速']
        else:
            pre_v = df.iloc[idx]['GPS车速']
            
#         print('idx:', idx, '  ', 'tag:', tag, '  ', '车速:', df.iloc[idx]['GPS车速'],'  ', 'pre_v:', pre_v,
#               '  ', 'feature_idx:', feature_idx,'  ', 'begin:', begin_index, '  ', 'end:', end_index,
#               '  ', '时间间隔:', df.iloc[idx]['时间间隔'])

            
    return feature_df

## 主函数

In [9]:
def main(df,flag):
    '''
    @func: 主函数，调用以上函数，并返回运动学片段数据
    @params: [df] 预训练数据
    @params: [flag] 所处理文件的标志
    @return: [feature_df] 运动学片段数据
    ''' 
    df = fillData(raw_data_1_df)
    df = moreAcce_and_lessDec(df)
    df = updateTrafficJamDate(df)
    df = more_180_Seconds_process(df)
    df.to_excel('./preprocess_' + str(flag) + '.xlsx')
    feature_df = searctKinematicSegment(df)
    feature_df = feature_df[feature_df['怠速时间']<180]
    return feature_df

In [10]:
feature_1 = main(raw_data_1_df, 1)
feature_2 = main(raw_data_2_df, 2)
feature_3 = main(raw_data_3_df, 3)

searctKinematicSegment:  10%|▉         | 18478/189234 [00:40<03:15, 873.66it/s]


searctKinematicSegment:  20%|██        | 38094/189234 [01:20<07:01, 358.33it/s]


searctKinematicSegment:  31%|███       | 57910/189234 [01:53<05:19, 410.60it/s]


searctKinematicSegment:  40%|████      | 75847/189234 [02:30<03:36, 522.98it/s]


searctKinematicSegment:  50%|█████     | 95284/189234 [03:10<02:51, 549.37it/s]


...

searctKinematicSegment:  59%|█████▊    | 111028/189234 [03:35<03:26, 378.45it/s]


searctKinematicSegment:  59%|█████▊    | 111140/189234 [03:35<02:45, 471.48it/s]


searctKinematicSegment:  59%|█████▉    | 111253/189234 [03:35<02:16, 570.78it/s]


searctKinematicSegment:  59%|█████▉    | 111367/189234 [03:35<01:56, 670.93it/s]


searctKinematicSegment:  59%|█████▉    | 111481/189234 [03:35<01:41, 764.97it/s]


searctKinematicSegment:  59%|█████▉    | 111584/189234 [03:36<05:32, 233.78it/s]


searctKinematicSegment:  59%|█████▉    | 111699/189234 [03:36<04:12, 307.08it/s]


searctKinematicSegment:  59%|█████▉    | 111804/189234 [03:37<04:52, 264.45it/s]


searctKinematicSegment:  59%|█████▉    | 111915/189234 [03:37<04:30, 285.36it/s]


searctKinematicSegment:  59%|█████▉    | 112077/189234 [03:37<03:23, 378.90it/s]


searctKinematicSegment:  59%|█████▉    | 112189/189234 [03:37<02:43, 472.59it/s]


searctKinematicSegment:  59%|█████▉    | 112293/189234 [03:38<02:16, 563.91it/s]


sear

searctKinematicSegment:  67%|██████▋   | 127693/189234 [04:11<02:42, 379.28it/s]


searctKinematicSegment:  68%|██████▊   | 127745/189234 [04:11<02:40, 383.35it/s]


searctKinematicSegment:  68%|██████▊   | 127878/189234 [04:11<02:05, 487.03it/s]


searctKinematicSegment:  68%|██████▊   | 127959/189234 [04:11<02:45, 370.24it/s]


searctKinematicSegment:  68%|██████▊   | 128043/189234 [04:11<02:35, 393.34it/s]


searctKinematicSegment:  68%|██████▊   | 128111/189234 [04:12<02:42, 375.48it/s]


searctKinematicSegment:  68%|██████▊   | 128188/189234 [04:12<02:17, 443.45it/s]


searctKinematicSegment:  68%|██████▊   | 128303/189234 [04:12<01:52, 543.23it/s]


searctKinematicSegment:  68%|██████▊   | 128378/189234 [04:12<02:48, 361.70it/s]


searctKinematicSegment:  68%|██████▊   | 128512/189234 [04:13<02:46, 365.61it/s]


searctKinematicSegment:  68%|██████▊   | 128662/189234 [04:13<02:08, 472.61it/s]


searctKinematicSegment:  68%|██████▊   | 128743/189234 [04:13<02:55, 344.35it/s]


sear

searctKinematicSegment:  77%|███████▋  | 145371/189234 [04:46<01:40, 434.32it/s]


searctKinematicSegment:  77%|███████▋  | 145486/189234 [04:46<01:21, 533.89it/s]


searctKinematicSegment:  77%|███████▋  | 145559/189234 [04:46<01:17, 560.22it/s]


searctKinematicSegment:  77%|███████▋  | 145680/189234 [04:47<01:05, 667.70it/s]


searctKinematicSegment:  77%|███████▋  | 145814/189234 [04:47<00:55, 784.95it/s]


searctKinematicSegment:  77%|███████▋  | 145914/189234 [04:47<01:23, 519.13it/s]


searctKinematicSegment:  77%|███████▋  | 145993/189234 [04:47<01:25, 503.06it/s]


searctKinematicSegment:  77%|███████▋  | 146146/189234 [04:47<01:08, 629.63it/s]


searctKinematicSegment:  77%|███████▋  | 146271/189234 [04:48<01:22, 519.45it/s]


searctKinematicSegment:  77%|███████▋  | 146386/189234 [04:48<01:08, 621.49it/s]


searctKinematicSegment:  77%|███████▋  | 146495/189234 [04:48<00:59, 712.45it/s]


searctKinematicSegment:  77%|███████▋  | 146590/189234 [04:48<01:48, 394.60it/s]


sear

searctKinematicSegment:  86%|████████▋ | 163303/189234 [05:24<01:13, 353.72it/s]


searctKinematicSegment:  86%|████████▋ | 163369/189234 [05:24<01:25, 303.89it/s]


searctKinematicSegment:  86%|████████▋ | 163473/189234 [05:24<01:06, 385.52it/s]


searctKinematicSegment:  86%|████████▋ | 163602/189234 [05:24<00:52, 488.11it/s]


searctKinematicSegment:  87%|████████▋ | 163701/189234 [05:24<00:44, 575.08it/s]


searctKinematicSegment:  87%|████████▋ | 163795/189234 [05:25<00:39, 650.62it/s]


searctKinematicSegment:  87%|████████▋ | 163884/189234 [05:25<01:25, 296.06it/s]


searctKinematicSegment:  87%|████████▋ | 163950/189234 [05:25<01:21, 310.26it/s]


searctKinematicSegment:  87%|████████▋ | 164007/189234 [05:26<01:17, 325.91it/s]


searctKinematicSegment:  87%|████████▋ | 164058/189234 [05:26<01:15, 331.83it/s]


searctKinematicSegment:  87%|████████▋ | 164104/189234 [05:26<01:13, 340.74it/s]


searctKinematicSegment:  87%|████████▋ | 164213/189234 [05:26<00:58, 429.12it/s]


sear

searctKinematicSegment:  95%|█████████▌| 180643/189234 [05:59<00:10, 799.08it/s]


searctKinematicSegment:  96%|█████████▌| 180742/189234 [06:00<00:18, 469.74it/s]


searctKinematicSegment:  96%|█████████▌| 180872/189234 [06:00<00:14, 580.59it/s]


searctKinematicSegment:  96%|█████████▌| 180965/189234 [06:00<00:20, 399.83it/s]


searctKinematicSegment:  96%|█████████▌| 181068/189234 [06:00<00:16, 489.37it/s]


searctKinematicSegment:  96%|█████████▌| 181149/189234 [06:00<00:21, 373.49it/s]


searctKinematicSegment:  96%|█████████▌| 181267/189234 [06:01<00:16, 469.23it/s]


searctKinematicSegment:  96%|█████████▌| 181347/189234 [06:01<00:23, 341.29it/s]


searctKinematicSegment:  96%|█████████▌| 181435/189234 [06:01<00:24, 313.41it/s]


searctKinematicSegment:  96%|█████████▌| 181487/189234 [06:01<00:22, 348.27it/s]


searctKinematicSegment:  96%|█████████▌| 181558/189234 [06:01<00:18, 411.03it/s]


searctKinematicSegment:  96%|█████████▌| 181669/189234 [06:02<00:14, 506.71it/s]


sear

In [11]:
feature_1.to_excel('./feature_1.xlsx')
feature_2.to_excel('./feature_2.xlsx')
feature_3.to_excel('./feature_3.xlsx')